In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(), # convert to tensor and scale pixels from [0, 255] to [0, 1] 
    transforms.Normalize((0.1307,), (0.3081,)) # normalize the data, these numbers are the mean and std of the MNIST dataset
])

In [4]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

100.0%
100.0%
100.0%
100.0%


In [ ]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784, 128), # 28*28 = 784, because the images are 28x28 pixels, converting it to a 1D(1,784) array from a 2D(1,28,28) array
            nn.ReLU(),
            nn.Linear(128, 10)# 10 is the number of output classes, 0-9
        )
    
    def forward(self, x): # flow of data through the network
        x = self.flatten(x)
        x = self.layers(x)
        return x

In [6]:
# check if GPU is available
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# initialize model and move it to the device
model = MNISTClassifier().to(device)

# initialize loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cpu


In [17]:
def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        #track metrics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100 * correct / total
            print(f"Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")
            running_loss = 0.0
            correct = 0
            total = 0
            

In [18]:
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy
    

In [19]:
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    # training model
    train_epoch(model, train_loader, loss_function, optimizer, device)
    
    # evaluate model
    accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy: {accuracy:.2f}%")

Epoch 1/10
Loss: 0.0083 | Accuracy: 99.66%
Loss: 0.0042 | Accuracy: 99.86%
Loss: 0.0094 | Accuracy: 99.69%
Loss: 0.0055 | Accuracy: 99.83%
Loss: 0.0031 | Accuracy: 99.92%
Loss: 0.0058 | Accuracy: 99.78%
Loss: 0.0095 | Accuracy: 99.70%
Loss: 0.0048 | Accuracy: 99.88%
Loss: 0.0055 | Accuracy: 99.88%
Test Accuracy: 97.85%
Epoch 2/10
Loss: 0.0043 | Accuracy: 99.88%
Loss: 0.0038 | Accuracy: 99.88%
Loss: 0.0029 | Accuracy: 99.94%
Loss: 0.0055 | Accuracy: 99.84%
Loss: 0.0151 | Accuracy: 99.55%
Loss: 0.0151 | Accuracy: 99.50%
Loss: 0.0120 | Accuracy: 99.66%
Loss: 0.0089 | Accuracy: 99.72%
Loss: 0.0078 | Accuracy: 99.73%
Test Accuracy: 97.80%
Epoch 3/10
Loss: 0.0086 | Accuracy: 99.75%
Loss: 0.0067 | Accuracy: 99.72%
Loss: 0.0067 | Accuracy: 99.78%
Loss: 0.0065 | Accuracy: 99.78%
Loss: 0.0080 | Accuracy: 99.77%
Loss: 0.0047 | Accuracy: 99.84%
Loss: 0.0108 | Accuracy: 99.62%
Loss: 0.0099 | Accuracy: 99.70%
Loss: 0.0096 | Accuracy: 99.67%
Test Accuracy: 97.89%
Epoch 4/10
Loss: 0.0043 | Accuracy: 9